In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import pickle

In [6]:
df = pd.read_csv("C:/Users/dharn/SWE/final_merged_data.csv")

In [8]:
df.isnull().sum()

last_reported                        0
station_id                           0
num_bikes_available                  0
num_docks_available                  0
is_installed                         0
                                    ..
max_barometric_pressure_hpa          0
min_pressure_quality_indicator       0
min_barometric_pressure_hpa          0
pressure_std_quality_indicator       0
barometric_pressure_std_deviation    0
Length: 78, dtype: int64

In [7]:
df.dropna(inplace = True)

,last_reported,station_id,num_bikes_available,num_docks_available,is_installed,is_renting,is_returning,name,address,lat,...,min_humidity_quality_indicator,min_relative_humidity_percent,humidity_std_quality_indicator,relative_humidity_std_deviation,max_pressure_quality_indicator,max_barometric_pressure_hpa,min_pressure_quality_indicator,min_barometric_pressure_hpa,pressure_std_quality_indicator,barometric_pressure_std_deviation
0,2024-12-01 00:10:00,10,15,1,True,True,True,DAME STREET,Dame Street,53.344006,...,0,83.2,0,0.284,0,1002.56,0,1002.26,0,0.083
1,2024-12-01 00:10:00,100,17,8,True,True,True,HEUSTON BRIDGE (SOUTH),Heuston Bridge (South),53.347107,...,0,83.2,0,0.284,0,1002.56,0,1002.26,0,0.083
2,2024-12-01 00:10:00,109,20,9,True,True,True,BUCKINGHAM STREET LOWER,Buckingham Street Lower,53.353333,...,0,83.2,0,0.284,0,1002.56,0,1002.26,0,0.083
3,2024-12-01 00:10:00,11,1,29,True,True,True,EARLSFORT TERRACE,Earlsfort Terrace,53.334293,...,0,83.2,0,0.284,0,1002.56,0,1002.26,0,0.083
4,2024-12-01 00:10:00,114,4,36,True,True,True,WILTON TERRACE (PARK),Wilton Terrace (Park),53.333652,...,0,83.2,0,0.284,0,1002.56,0,1002.26,0,0.083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298941,2024-12-31 23:50:00,83,33,7,True,True,True,EMMET ROAD,Emmet Road,53.340714,...,0,90.5,0,0.299,0,997.15,0,997.06,0,0.022
298942,2024-12-31 23:50:00,84,23,7,True,True,True,BROOKFIELD ROAD,Brookfield Road,53.339005,...,0,90.5,0,0.299,0,997.15,0,997.06,0,0.022
298943,2024-12-31 23:50:00,90,2,38,True,True,True,BENSON STREET,Benson Street,53.344154,...,0,90.5,0,0.299,0,997.15,0,997.06,0,0.022
298944,2024-12-31 23:50:00,95,25,15,True,True,True,ROYAL HOSPITAL,Royal Hospital,53.343900,...,0,90.5,0,0.299,0,997.15,0,997.06,0,0.022


In [10]:
unique_counts = df.nunique()
unique_counts

last_reported                        4433
station_id                            115
num_bikes_available                    41
num_docks_available                    41
is_installed                            1
                                     ... 
max_barometric_pressure_hpa          2470
min_pressure_quality_indicator          1
min_barometric_pressure_hpa          2516
pressure_std_quality_indicator          1
barometric_pressure_std_deviation     211
Length: 78, dtype: int64

Dropping unwanted columns

In [13]:
constant_cols = unique_counts[unique_counts == 1].index
keep_cols = ["year", "month"]
cols_to_drop = constant_cols.difference(keep_cols)
df.drop(columns=cols_to_drop, inplace=True)
print("Dropped columns:", cols_to_drop)

Dropped columns: Index(['air_temp_std_quality_indicator',
       'earth_temp_std_100cm_quality_indicator',
       'earth_temp_std_30cm_quality_indicator',
       'earth_temp_std_50cm_quality_indicator',
       'grass_temp_std_quality_indicator', 'humidity_std_quality_indicator',
       'is_installed', 'is_renting', 'is_returning',
       'max_air_temp_quality_indicator',
       'max_earth_temp_100cm_quality_indicator',
       'max_earth_temp_30cm_quality_indicator',
       'max_earth_temp_50cm_quality_indicator',
       'max_grass_temp_quality_indicator', 'max_humidity_quality_indicator',
       'max_pressure_quality_indicator',
       'max_soil_temp_10cm_quality_indicator',
       'max_soil_temp_20cm_quality_indicator',
       'max_soil_temp_5cm_quality_indicator', 'min_air_temp_quality_indicator',
       'min_earth_temp_100cm_quality_indicator',
       'min_earth_temp_30cm_quality_indicator',
       'min_earth_temp_50cm_quality_indicator',
       'min_grass_temp_quality_indicator', '

In [15]:
df.columns

Index(['last_reported', 'station_id', 'num_bikes_available',
       'num_docks_available', 'name', 'address', 'lat', 'lon', 'capacity',
       'year', 'month', 'day', 'hour', 'minute', 'max_air_temperature_celsius',
       'min_air_temperature_celsius', 'air_temperature_std_deviation',
       'max_grass_temperature_celsius', 'min_grass_temperature_celsius',
       'grass_temperature_std_deviation', 'max_soil_temperature_5cm_celsius',
       'min_soil_temperature_5cm_celsius',
       'soil_temperature_std_deviation_5cm',
       'max_soil_temperature_10cm_celsius',
       'min_soil_temperature_10cm_celsius',
       'soil_temperature_std_deviation_10cm',
       'max_soil_temperature_20cm_celsius',
       'min_soil_temperature_20cm_celsius',
       'soil_temperature_std_deviation_20cm',
       'max_earth_temperature_30cm_celsius',
       'min_earth_temperature_30cm_celsius',
       'earth_temperature_std_deviation_30cm',
       'max_earth_temperature_50cm_celsius',
       'min_earth_temper

In [17]:
unwanted_cols = ['lat', 'lon', 'air_temperature_std_deviation', 'max_grass_temperature_celsius','min_grass_temperature_celsius',
       'grass_temperature_std_deviation', 'max_soil_temperature_5cm_celsius','min_soil_temperature_5cm_celsius','soil_temperature_std_deviation_5cm',
       'max_soil_temperature_10cm_celsius',
       'min_soil_temperature_10cm_celsius',
       'soil_temperature_std_deviation_10cm',
       'max_soil_temperature_20cm_celsius',
       'min_soil_temperature_20cm_celsius',
       'soil_temperature_std_deviation_20cm',
       'max_earth_temperature_30cm_celsius',
       'min_earth_temperature_30cm_celsius',
       'earth_temperature_std_deviation_30cm',
       'max_earth_temperature_50cm_celsius',
       'min_earth_temperature_50cm_celsius',
       'earth_temperature_std_deviation_50cm',
       'max_earth_temperature_100cm_celsius',
       'min_earth_temperature_100cm_celsius',
       'earth_temperature_std_deviation_100cm','relative_humidity_std_deviation', 'max_barometric_pressure_hpa',
       'min_barometric_pressure_hpa', 'barometric_pressure_std_deviation']

df.drop(columns=unwanted_cols, inplace=True)

In [19]:
df.columns

Index(['last_reported', 'station_id', 'num_bikes_available',
       'num_docks_available', 'name', 'address', 'capacity', 'year', 'month',
       'day', 'hour', 'minute', 'max_air_temperature_celsius',
       'min_air_temperature_celsius', 'max_relative_humidity_percent',
       'min_relative_humidity_percent'],
      dtype='object')

In [21]:
df.dtypes

last_reported                     object
station_id                         int64
num_bikes_available                int64
num_docks_available                int64
name                              object
address                           object
capacity                           int64
year                               int64
month                              int64
day                                int64
hour                               int64
minute                             int64
max_air_temperature_celsius      float64
min_air_temperature_celsius      float64
max_relative_humidity_percent    float64
min_relative_humidity_percent    float64
dtype: object

date conversion

In [23]:
df['last_reported'] = pd.to_datetime(df['last_reported'], format='%Y-%m-%d %H:%M:%S')

In [25]:
df.dtypes

last_reported                    datetime64[ns]
station_id                                int64
num_bikes_available                       int64
num_docks_available                       int64
name                                     object
address                                  object
capacity                                  int64
year                                      int64
month                                     int64
day                                       int64
hour                                      int64
minute                                    int64
max_air_temperature_celsius             float64
min_air_temperature_celsius             float64
max_relative_humidity_percent           float64
min_relative_humidity_percent           float64
dtype: object

In [27]:
df['temperature'] = (df["max_air_temperature_celsius"] + df["min_air_temperature_celsius"])/2
df['humidity'] = (df["max_relative_humidity_percent"] + df["min_relative_humidity_percent"])/2

In [29]:
drop_cols = ["max_air_temperature_celsius",
"min_air_temperature_celsius",
"max_relative_humidity_percent",    
"min_relative_humidity_percent"]
df.drop(columns=drop_cols, inplace=True)

In [31]:
df.head(5)

,last_reported,station_id,num_bikes_available,num_docks_available,name,address,capacity,year,month,day,hour,minute,temperature,humidity
0,2024-12-01 00:10:00,10,15,1,DAME STREET,Dame Street,16,2024,12,1,0,10,13.955,83.75
1,2024-12-01 00:10:00,100,17,8,HEUSTON BRIDGE (SOUTH),Heuston Bridge (South),25,2024,12,1,0,10,13.955,83.75
2,2024-12-01 00:10:00,109,20,9,BUCKINGHAM STREET LOWER,Buckingham Street Lower,29,2024,12,1,0,10,13.955,83.75
3,2024-12-01 00:10:00,11,1,29,EARLSFORT TERRACE,Earlsfort Terrace,30,2024,12,1,0,10,13.955,83.75
4,2024-12-01 00:10:00,114,4,36,WILTON TERRACE (PARK),Wilton Terrace (Park),40,2024,12,1,0,10,13.955,83.75


Creating new feature "day_of_week"

In [35]:
df['day_of_week'] = df['last_reported'].dt.dayofweek

In [39]:
df.head(5)

,last_reported,station_id,num_bikes_available,num_docks_available,name,address,capacity,year,month,day,hour,minute,temperature,humidity,day_of_week
0,2024-12-01 00:10:00,10,15,1,DAME STREET,Dame Street,16,2024,12,1,0,10,13.955,83.75,6
1,2024-12-01 00:10:00,100,17,8,HEUSTON BRIDGE (SOUTH),Heuston Bridge (South),25,2024,12,1,0,10,13.955,83.75,6
2,2024-12-01 00:10:00,109,20,9,BUCKINGHAM STREET LOWER,Buckingham Street Lower,29,2024,12,1,0,10,13.955,83.75,6
3,2024-12-01 00:10:00,11,1,29,EARLSFORT TERRACE,Earlsfort Terrace,30,2024,12,1,0,10,13.955,83.75,6
4,2024-12-01 00:10:00,114,4,36,WILTON TERRACE (PARK),Wilton Terrace (Park),40,2024,12,1,0,10,13.955,83.75,6


In [41]:
df.to_csv('bike_weather_data.csv', index = False)

Model Creation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor   # CHANGED: Using RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import pickle

# Load the cleaned dataset. Adjust the file path if necessary.
data = pd.read_csv("bike_weather_data.csv")

# Group the data by the relevant fields.
grouped = data.groupby(['name', 'station_id', 'year', 'month', 'day', 'hour', 'minute']).agg({
    'capacity': 'first',
    'temperature': 'mean',
    'humidity': 'mean',
    'num_bikes_available': 'mean',
    'num_docks_available': 'mean',
    'day_of_week': 'first'
}).reset_index()

# Make sure occupancy (num_docks_available) is an integer
grouped['num_docks_available'] = grouped['num_docks_available'].round().astype(int)

# Compute historical averages (will be used as fallback or dynamic model input)
avg_docks = grouped.groupby('station_id')['num_docks_available'].mean().to_dict()
avg_capacity = grouped.groupby('station_id')['capacity'].mean().to_dict()

# Define the features and target variable. Here we use raw features.
features = [
    'station_id',
    'year',
    'day_of_week',
    'num_docks_available',
    'capacity',
    'temperature', 
    'humidity'
]
target = 'num_bikes_available'

# Create X and Y datasets.
X = grouped[features]
Y = grouped[target]

# Split the data into training and testing sets.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# CHANGED: Train a Random Forest model instead of Linear Regression.
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)

# Predict on the test set.
Y_pred = model.predict(X_test)

# Evaluate model performance.
mae = mean_absolute_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)
print("Model Performance on Station Averages:")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")

# Save the model using joblib and pickle.
joblib.dump(model, "bike_station_rf_model.joblib")
with open("bike_station_rf_model.pkl", "wb") as f:
    pickle.dump(model, f)
print("\n✅ Model saved as:")
print("- bike_station_rf_model.pkl")
print("- bike_station_rf_model.joblib")

# Save historical occupancy data.
with open("avg_docks.pkl", "wb") as f:
    pickle.dump(avg_docks, f)
with open("avg_capacity.pkl", "wb") as f:
    pickle.dump(avg_capacity, f)
print("\n✅ Historical average data saved (avg_docks.pkl and avg_capacity.pkl)")
